# Search for replicas

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
from apps.mcmasters.replicawork import ReplicaWorkchain
from aiida.work.process import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview
import time
import ase.io
import ase.units as aseu
import urlparse
import numpy as np

import matplotlib.pyplot as plt
from pprint import pprint

from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return '<img width="400px" src="data:image/png;base64,{}" title="">'.format(th)
def html_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

In [ ]:
preproc_v = 0.02
def preprocess(replica):
    global preproc_v
    struc_ase = []
    with out_preproc:
        style = {'description_width': '120px'}
        layout = {'width': '70%'}
        progress = ipw.FloatProgress(description='Parsing images...', min=0, max=1, value=0.,
                                     style=style, layout=layout)
        display(progress)

    for i, rep in enumerate(replica):
        with out_preproc:
            progress.value = (i+1.)/len(replica)
        prepoc_failed = False
                
        struc = rep['struc_out'].get_ase()
        struc_ase.append(struc)
        
        # figure out the colvar_type
        if rep['colvar'].keys()[0] == 'DISTANCE':
            colvar_type = 'DISTANCE'
        elif rep['colvar'].keys()[0] == 'ANGLE_PLANE_PLANE':
            colvar_type = 'ANGLE_PLANE_PLANE'
        else:
            colvar_type = 'NOT IMPLEMENTED'

        # replace the target atoms with O
        t = struc
        if colvar_type == 'DISTANCE':
            # list of atoms to change
            atoms_list = [int(x) for x in rep['colvar']['DISTANCE']['ATOMS'].split()]
            # collective variable
            if len(atoms_list) == 2:
                # distance of the two atoms
                colvar_actual = t[atoms_list[0]-1].position-t[atoms_list[1]-1].position
                colvar_actual = np.linalg.norm(colvar_actual)
            else:
                colvar_actual = 0
                prepoc_failed = True
                print('More than two atoms defined by DISTANCE colvar!')
        elif colvar_type == 'ANGLE_PLANE_PLANE':
            atoms_list = []
            vector = []
            plane1, plane2 = (rep['colvar']['ANGLE_PLANE_PLANE']['PLANE  '],
                              rep['colvar']['ANGLE_PLANE_PLANE']['PLANE'])
            for p in (plane1, plane2):
                if p['DEF_TYPE'] == 'ATOMS':
                    atoms_list.append([int(x) for x in p['ATOMS'].split()])
                if p['DEF_TYPE'] == 'VECTOR':
                    vector.append([np.float(x) for x in p['NORMAL_VECTOR'].split()])

            atoms_list = atoms_list[0]  # because in principle there can be two each
            vector = vector[0]
            # TODO: This assumes one plane defined by atoms
            #       and one plane defined by a normal vector.
            #       This can be generalized to two planes
            #       defined by atoms.
            v1 = t[atoms_list[0]-1].position - t[atoms_list[1]-1].position
            v2 = t[atoms_list[0]-1].position - t[atoms_list[2]-1].position
            first_cross = np.cross(-v1, v2)
            
            dotp = np.dot(first_cross, vector)
            cosine = dotp/(np.linalg.norm(first_cross)*np.linalg.norm(vector))
            angle = np.arccos(cosine)*180./np.pi  # in deg
            #print 'Cross: {}'.format(first_cross)
            #print 'dotp: {}'.format(dotp)
            #print 'Cosine: {}'.format(cosine)
            #print 'Angle: {}'.format(angle)
            colvar_actual = angle

        # and the actual replacing
        for a in atoms_list:
            t[a-1].symbol = 'O'

        thumbnail = render_thumbnail(t)
        
        # try calculating the distance to the previous replica
        try:
            d2next = get_replica_distance(struc_ase[i-1], struc)
        except IndexError:
            d2next = '-'
        
        struc_extras = {
            'preproc_v': preproc_v,
            'preproc_failed': prepoc_failed,
            'thumbnail': thumbnail,
            'd2next': d2next,
            'colvar_actual': colvar_actual
        }
        rep['struc_out'].description = rep['cp2k_desc']
        rep['struc_out'].set_extras(struc_extras)
        rep['cp2k'].set_extras({
            'preproc_v': preproc_v,
            'preproc_failed': prepoc_failed,
        })
    
    return replica

In [ ]:
def get_replicas(preproc = False):
    global preproc_v
    qb = QueryBuilder()

    qb.append(
        WorkCalculation,
        tag='wc'
    )

    cp2k_filters = {
        'label': {'==': 'replica_geo_opt'},
        'description': {'like': 'replica_%'},
    }
    
    if preproc:
        cp2k_filters['and'] = [
            {'extras': {'has_key': 'preproc_v'}},
            {'extras.preproc_v': {'==', preproc_v}},
            {'extras.preproc_failed': {'==': False}}
        ]
    else:
        cp2k_filters['or'] = [
            {'extras': {'!has_key': 'preproc_v'}},
            {'extras.preproc_v': {'<': preproc_v}}
        ]

    qb.append(
        Cp2kCalculation,
        filters=cp2k_filters,
        tag='cp2k',
        output_of='wc',
        project='*'
    )

    qb.append(
        StructureData,
        output_of='cp2k',
        project='*',
    )

    qb.append(
        ParameterData,
        input_of='cp2k',
        project=['attributes.FORCE_EVAL',
                 'attributes.MOTION.CONSTRAINT.COLLECTIVE'],
        filters={
            'attributes': {'has_key': 'FORCE_EVAL'}
        },
        tag='parameter_in'
    )

    qb.append(
        ParameterData,
        output_of='cp2k',
        filters={
            'attributes.exceeded_walltime': {'==': False}
        },
        project=['attributes.energy', 'ctime'],
        tag='parameter_output'
    )

    qb.order_by({'cp2k': {'ctime': 'asc'}})

    return qb.all()

In [ ]:
def replica_parse_desc(r):
    def find_colvar(force_eval):
        # return the force_eval.subsys.colvar
        try:  # if force_eval is a dictionary...
            return force_eval['SUBSYS']['COLVAR']
        except TypeError:  # or look it up in the list
            for f in force_eval:
                if f['SUBSYS'].has_key('COLVAR'):
                    return f['SUBSYS']['COLVAR']
        else:
            raise('Something went wrong looking for COLVAR!')

    import re
    from collections import OrderedDict
    pattern = r'replica_(.+)_(.+)'
    names_count = OrderedDict()
    for rep in r:
        # the cp2kcalculation.description is LIKE 'replica_%(name)_%(target)
        name = re.findall(pattern, rep[0].description)[0][0]
        #print name
        colvar = find_colvar(rep[2])
        # put it together
        try:
            names_count[name].append({
                'cp2k_desc': rep[0].description,
                'struc_out': rep[1],
                'colvar': colvar,
                'collective': rep[3],
                'energy': rep[4],
                'datetime': rep[5],
                'cp2k': rep[0]
            })
        except KeyError:
            names_count[name] = [{
                'cp2k_desc': rep[0].description,
                'struc_out': rep[1],
                'colvar': colvar,
                'collective': rep[3],
                'energy': rep[4],
                'datetime': rep[5],
                'cp2k': rep[0]
            },]
        #print (name, target)
    return names_count

def get_replica_distance(s1, s2):
    a1 = s1.get_positions()
    a2 = s2.get_positions()
    return np.linalg.norm(a1-a2)

In [ ]:
print('Retrieving unparsed NEB calculations...')
replicas_not = replica_parse_desc(get_replicas(False))
print('Preprocessing {} replicas...'.format(len(replicas_not.keys())))
out_preproc_text = ipw.Output()
out_preproc = ipw.Output()
display(out_preproc_text, out_preproc)

for i, k in enumerate(replicas_not.keys()):
    with out_preproc_text:
        print('{}: {}/{}'.format(k, i+1, len(replicas_not.keys())))
    with out_preproc:
        clear_output()
        preprocess(replicas_not[k])

with out_preproc:
    clear_output()
with out_preproc_text:
    clear_output()

print('Done!')

In [ ]:
replica = replica_parse_desc(get_replicas(True))
keys, date = replica.keys(), [str(replica[rep][0]['datetime']) for rep in replica.keys()]
print 'Found replicas: '
for x in zip(keys, date):
    print '{} - {}'.format(x[0], x[1])

drop_replica = ipw.Dropdown(options = keys,
                            description = 'Replica: ',
                            index = len(keys)-1)
display(drop_replica)

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

text_name = ipw.Text(placeholder='Replica name',
                     description='Replica name',
                     style=style, layout=layout)

def on_submit(b):
    replica_name = drop_replica.value
    this_replica = replica[replica_name]
    
    if this_replica[0]['colvar'].keys()[0] == 'DISTANCE':
        colvar_type = 'DISTANCE'
    elif this_replica[0]['colvar'].keys()[0] == 'ANGLE_PLANE_PLANE':
        colvar_type = 'ANGLE_PLANE_PLANE'
    else:
        colvar_type = 'NOT IMPLEMENTED'

    show_th = show_thumbnails(this_replica)

    with output_thumbnails:
        clear_output()
        html = 'COLVAR: {}'.format(colvar_type)
        html += show_th['html']        
        display(ipw.HTML(html))

    with output_plot:
        clear_output()
        plt.figure(figsize=(10, 5))
        plt.ylabel('Energy/eV')
        plt.xlabel('Collective variable')
        plt.plot(show_th['plot'][0], [x*aseu.Hartree for x in show_th['plot'][1]], 'o-')
        plt.grid()
        plt.show()
        
    with output_header:
        clear_output()
        html = '<h2>{}</h2><br>'.format(replica_name)
        display(ipw.HTML(html))

output_thumbnails = ipw.Output()
output_plot = ipw.Output()
output_header = ipw.Output()
btn_submit = ipw.Button(description="Show")
btn_submit.on_click(on_submit)
display(btn_submit, output_header, output_plot, output_thumbnails)

In [ ]:
def show_thumbnails(replica):
    plot_energy = []
    plot_colvar = []
    
    html = '<table>'
    for i, rep in enumerate(replica):
        thumbnail = rep['struc_out'].get_extra('thumbnail')
        d2next = rep['struc_out'].get_extra('d2next')
        colvar_actual = rep['struc_out'].get_extra('colvar_actual')
        
        # find out which kind of colvar we are dealing with
        if rep['colvar'].keys()[0] == 'DISTANCE':
            colvar_type = 'DISTANCE'
            colvar_target = rep['collective']['TARGET']
        elif rep['colvar'].keys()[0] == 'ANGLE_PLANE_PLANE':
            colvar_type = 'ANGLE_PLANE_PLANE'
            colvar_target = rep['collective']['TARGET']

        plot_energy.append(rep['energy'])
        plot_colvar.append(colvar_actual)

        # The table cell
        if i%2 == 0:
            html += '<tr>'
        html += '<td><img width="400px" src="data:image/png;base64,{}" title="">'.format(thumbnail)

        # Output some information about the replica...
        html += '<p><b>Target: {}</b> ({})<br> <b>Energy:</b> {}<br> <b>d2prev:</b> {}</p>'\
                .format(colvar_target.split()[1], colvar_actual, rep['energy'], d2next)
        html += '<p>pk: {}</p>'.format(rep['struc_out'].pk)
        # ... and the download link.
        html += '<p><a target="_blank" href="./export_structure.ipynb?uuid={}">Export Structure</a></p><td>'\
                .format(rep['struc_out'].uuid)
    
        if i%2 == 1:
            html += '</tr>'

    html += '</tr>'
    html += '</table>'
    
    # return the html and plot
    return {
        'html': html,
        'plot': (plot_colvar, plot_energy)
    }

In [ ]:
def show_ngl(replica): # don't
    the_views = []
    for i, rep in enumerate(replica):
        the_views.append(ipw.Output())
        display(the_views[-1])
        with the_views[-1]:
            clear_output()
            display(nglview.show_ase(rep[1].get_ase()))

        #print rep[0], rep[1], rep[2]